In [1]:
import mysql.connector  as connection
import pandas as pd
import datetime as dt
import os

In [2]:
file_dir = 'C:/Users/User/Desktop/Moje dokumenty/Raporty obce/Kacper/63_issue_158946_payments/Excel/2022/'
db_credentials_dir = 'C:/Users/User/Desktop/Moje dokumenty/Raporty obce/Kacper/63_issue_158946_payments/'

In [3]:
all_books_22 = pd.read_csv(file_dir + 'all_books_22.csv')

C:\Users\User\AppData\Local\Temp\ipykernel_2112\1484331314.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  all_books_22 = pd.read_csv(file_dir + 'all_books_22.csv')


In [4]:
FACT_all_transactions = pd.read_csv(file_dir + 'FACT_all_transactions.csv')

C:\Users\User\AppData\Local\Temp\ipykernel_2112\3746988900.py:1: DtypeWarning: Columns (2,14) have mixed types. Specify dtype option on import or set low_memory=False.
  FACT_all_transactions = pd.read_csv(file_dir + 'FACT_all_transactions.csv')


In [5]:
FACT_all_transactions['Transaction_Date'] = pd.to_datetime(FACT_all_transactions['Transaction_Date'])

In [6]:
FACT_all_transactions['Selling_account'] = FACT_all_transactions['Selling_account'].astype(str)
FACT_all_transactions['Selling_account'] = FACT_all_transactions['Selling_account'].astype(str).replace(r'\.0', '', regex=True)

In [7]:
FACT_all_transactions_2022= FACT_all_transactions[(FACT_all_transactions['Transaction_Date'] >= pd.Timestamp('2022-01-01')) & (FACT_all_transactions['Transaction_Date'] < pd.Timestamp('2023-01-01')) ]

In [8]:
DIM_all_auction = pd.read_csv(file_dir + 'DIM_all_auction.csv')

In [10]:
FACT_all_transactions_2022[['auction_number', 'txnid']] = FACT_all_transactions_2022['Main auction'].str.split('_', expand=True)

C:\Users\User\AppData\Local\Temp\ipykernel_2112\145284229.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FACT_all_transactions_2022[['auction_number', 'txnid']] = FACT_all_transactions_2022['Main auction'].str.split('_', expand=True)
C:\Users\User\AppData\Local\Temp\ipykernel_2112\145284229.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FACT_all_transactions_2022[['auction_number', 'txnid']] = FACT_all_transactions_2022['Main auction'].str.split('_', expand=True)


In [14]:
DIM_all_auction[['auction_number', 'txnid']] = DIM_all_auction['MAU_Main auction'].str.split('_', expand=True)

In [15]:
DIM_Company = DIM_all_auction[['auction_number', 'Company', 'Seller','Currency']]

In [16]:
DIM_Company.drop_duplicates(subset='auction_number', inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_2112\1269495965.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DIM_Company.drop_duplicates(subset='auction_number', inplace=True)


In [17]:
DIM_Company['auction_number'] = DIM_Company['auction_number'].astype(str)
DIM_Company['auction_number'] = DIM_Company['auction_number'].astype(str).replace(r'\.0', '', regex=True)
DIM_Company['auction_number'] = DIM_Company['auction_number'].str.strip()

FACT_all_transactions_2022['auction_number'] = FACT_all_transactions_2022['auction_number'].astype(str)
FACT_all_transactions_2022['auction_number'] = FACT_all_transactions_2022['auction_number'].astype(str).replace(r'\.0', '', regex=True)
FACT_all_transactions_2022['auction_number'] = FACT_all_transactions_2022['auction_number'].str.strip()

C:\Users\User\AppData\Local\Temp\ipykernel_2112\2913898193.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DIM_Company['auction_number'] = DIM_Company['auction_number'].astype(str)
C:\Users\User\AppData\Local\Temp\ipykernel_2112\2913898193.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DIM_Company['auction_number'] = DIM_Company['auction_number'].astype(str).replace(r'\.0', '', regex=True)
C:\Users\User\AppData\Local\Temp\ipykernel_2112\2913898193.py:3: SettingWithCopyWarning: 
A value is trying to

In [18]:
transactions_2022_company = pd.merge(FACT_all_transactions_2022, DIM_Company, on='auction_number', how='left')

In [26]:
transactions_2022_company_exp = transactions_2022_company[transactions_2022_company['Transaction_exported'] > 0]

In [28]:
aggregation = {
    'Amount_Netto(local)': 'sum',
    'Currency': 'first',
    'Selling_account' : 'first',
    'Company' : 'first',
    'Seller' : 'first'
}

In [41]:
auctions_sum = transactions_2022_company_exp.groupby("auction_number").agg(aggregation).reset_index()


In [46]:
all_books_22['auction_number'] = all_books_22['auction_number'].astype(str)
all_books_22['auction_number'] = all_books_22['auction_number'].astype(str).replace(r'\.0', '', regex=True)
all_books_22['auction_number'] = all_books_22['auction_number'].str.strip()

In [51]:
aggregation_books = {
    'Amount_Netto(local)': 'sum',
    'Text': 'first',
    'Source.Name' : 'first',
    'type_split' : 'first',
}

In [52]:
books_sum = all_books_22.groupby("auction_number").agg(aggregation_books).reset_index()


In [62]:
books_sum.to_csv(file_dir + 'Accounts_Power_BI/books_sum.csv', index=False)
auctions_sum.to_csv(file_dir + 'Accounts_Power_BI/auctions_sum.csv', index=False)